# Lesson 6: Shipping as a web API

In [1]:
import "dotenv/config";

[Module: null prototype] { default: {} }

In [2]:
import { 
  loadAndSplitChunks, 
  initializeVectorstoreWithDocuments 
} from "./lib/helpers.ts";

const splitDocs = await loadAndSplitChunks({
  chunkSize: 1536,
  chunkOverlap: 128,
});

const vectorstore = await initializeVectorstoreWithDocuments({
  documents: splitDocs,
});

const retriever = vectorstore.asRetriever();

In [3]:
import { 
  createDocumentRetrievalChain, 
  createRephraseQuestionChain 
} from "./lib/helpers.ts";

const documentRetrievalChain = createDocumentRetrievalChain();
const rephraseQuestionChain = createRephraseQuestionChain();

In [4]:
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";

const ANSWER_CHAIN_SYSTEM_TEMPLATE = `You are an experienced researcher,
expert at interpreting and answering questions based on provided sources.
Using the below provided context and chat history, 
answer the user's question to the best of your ability
using only the resources provided. Be verbose!

<context>
{context}
</context>`;

const answerGenerationChainPrompt = ChatPromptTemplate.fromMessages([
  ["system", ANSWER_CHAIN_SYSTEM_TEMPLATE],
  new MessagesPlaceholder("history"),
  [
    "human", 
    `Now, answer this question using the previous context and chat history:
  
    {standalone_question}`
  ]
]);

In [5]:
import { 
  RunnablePassthrough, 
  RunnableSequence 
} from "@langchain/core/runnables";
import { ChatOpenAI } from "@langchain/openai";

const conversationalRetrievalChain = RunnableSequence.from([
  RunnablePassthrough.assign({
    standalone_question: rephraseQuestionChain,
  }),
  RunnablePassthrough.assign({
    context: documentRetrievalChain,
  }),
  answerGenerationChainPrompt,
  new ChatOpenAI({ modelName: "gpt-3.5-turbo-1106" }),
]);

In [6]:
import { HttpResponseOutputParser } from "langchain/output_parsers";

// "text/event-stream" is also supported
const httpResponseOutputParser = new HttpResponseOutputParser({
  contentType: "text/plain"
});

In [7]:
import { RunnableWithMessageHistory } from "@langchain/core/runnables"; 
import { ChatMessageHistory } from "langchain/stores/message/in_memory";

const messageHistory = new ChatMessageHistory();

const finalRetrievalChain = new RunnableWithMessageHistory({
  runnable: conversationalRetrievalChain,
  getMessageHistory: (_sessionId) => messageHistory,
  historyMessagesKey: "history",
  inputMessagesKey: "question",
}).pipe(httpResponseOutputParser);

Additionally, we'll want to bear in mind that users should not share chat histories, and we should create a new history object per session:

In [8]:
const messageHistories = {};

const getMessageHistoryForSession = (sessionId) => {
    if (messageHistories[sessionId] !== undefined) {
        return messageHistories[sessionId];
    } 
    const newChatSessionHistory = new ChatMessageHistory();
    messageHistories[sessionId] = newChatSessionHistory;
    return newChatSessionHistory;
};

We'll recreate our final chain with this new method:

In [9]:
const finalRetrievalChain = new RunnableWithMessageHistory({
  runnable: conversationalRetrievalChain,
  getMessageHistory: getMessageHistoryForSession,
  inputMessagesKey: "question",
  historyMessagesKey: "history",
}).pipe(httpResponseOutputParser);

In [10]:
const port = 8087;

In [11]:
const handler = async (request: Request): Response => {
  const body = await request.json();
  const stream = await finalRetrievalChain.stream({
    question: body.question
  }, { configurable: { sessionId: body.session_id } });

  return new Response(stream, { 
    status: 200,
    headers: {
      "Content-Type": "text/plain"
    },
  });
};

In [12]:
Deno.serve({ port }, handler);

Listening on http://localhost:8087/


{
  finished: Promise { <pending> },
  shutdown: [AsyncFunction: shutdown],
  ref: [Function: ref],
  unref: [Function: unref],
  [Symbol(Symbol.asyncDispose)]: [Function: [Symbol.asyncDispose]]
}

In [13]:
const decoder = new TextDecoder();

// readChunks() reads from the provided reader and yields the results into an async iterable
function readChunks(reader) {
  return {
    async* [Symbol.asyncIterator]() {
      let readResult = await reader.read();
      while (!readResult.done) {
        yield decoder.decode(readResult.value);
        readResult = await reader.read();
      }
    },
  };
}

const sleep = async () => {
  return new Promise((resolve) => setTimeout(resolve, 500));
}

In [14]:
const response = await fetch(`http://localhost:${port}`, {
    method: "POST",
    headers: {
        "content-type": "application/json",
    },
    body: JSON.stringify({
        question: "What are the prerequisites for this course?",
        session_id: "1", // Should randomly generate/assign
    })
});

// response.body is a ReadableStream
const reader = response.body?.getReader();

for await (const chunk of readChunks(reader)) {
  console.log("CHUNK:", chunk);
}

await sleep();

CHUNK: The requi
CHUNK: rements for the cours
CHUNK: e are desc
CHUNK: ribed in the context prov
CHUNK: ided. The instruc
CHUNK: tor mentions
CHUNK:  several p
CHUNK: rerequisite s
CHUNK: kills and knowledg
CHUNK: e areas that students 
CHUNK: should have before t
CHUNK: aking the course. 


CHUNK: Firstly, f
CHUNK: amiliarity wit
CHUNK: h basic proba
CHUNK: bility and st
CHUNK: atistics i
CHUNK: s assum
CHUNK: ed. The instructor specifi
CHUNK: es that most undergraduate s
CHUNK: tatistics classes, such as S
CHUNK: tat 116 at
CHUNK:  Stanford, will provide suffici
CHUNK: ent back
CHUNK: ground. A
CHUNK: dditionally, 
CHUNK: a basic understand
CHUNK: ing of linear algebra
CHUNK:  is required. U
CHUNK: ndergraduate courses like Math 51,
CHUNK:  103, Math 113,
CHUNK:  or CS205
CHUNK:  at Stanford are deemed to b
CHUNK: e sufficient prepar
CHUNK: ation in this area. K
CHUNK: nowledge of matrices, ve
CHUNK: ctors, matrix multiplic
CHUNK: ation, matrix inver
CHUNK: ses, and eigenvectors is e

In [15]:
const response = await fetch(`http://localhost:${port}`, {
  method: "POST",
  headers: {
    "content-type": "application/json",
  },
  body: JSON.stringify({
    question: "Can you list them in bullet point format?",
    session_id: "1", // Should randomly generate/assign
  })
});

// response.body is a ReadableStream
const reader = response.body?.getReader();

for await (const chunk of readChunks(reader)) {
  console.log("CHUNK:", chunk);
}

await sleep();

CHUNK: Certainly! Ba
CHUNK: sed on the 
CHUNK: information provi
CHUNK: ded, the prerequisi
CHUNK: tes for the course can
CHUNK:  be summ
CHUNK: arized a
CHUNK: s follows:

-
CHUNK:  Familiarity with ba
CHUNK: sic probabil
CHUNK: ity and sta
CHUNK: tistics,
CHUNK:  on par with most undergraduat
CHUNK: e statistics class
CHUNK: es like Stat 116 
CHUNK: at Stanford.
CHUNK: 
- Basic knowledge o
CHUNK: f linear a
CHUNK: lgebra, inc
CHUNK: luding underst
CHUNK: anding of ma
CHUNK: trices, vectors, matrix
CHUNK:  multiplic
CHUNK: ation, matrix invers
CHUNK: es, and eigenvectors
CHUNK: . Courses like 
CHUNK: Math 51,
CHUNK:  103, Math 11
CHUNK: 3, or CS205
CHUNK:  at Stanford are recommended as pr
CHUNK: eparation.
- Some
CHUNK:  programm
CHUNK: ing experience,
CHUNK:  with an emphasis
CHUNK:  on MATLAB or 
CHUNK: Octave. No C programming is requir
CHUNK: ed, and understanding bi
CHUNK: g O notatio
CHUNK: n and knowledge of data structures (e.g., linked lists, queues, binary trees) is more im

In [16]:
const response = await fetch(`http://localhost:${port}`, {
  method: "POST",
  headers: {
    "content-type": "application/json",
  },
  body: JSON.stringify({
    question: "What did I just ask you?",
    session_id: "2", // Should randomly generate/assign
  })
});

// response.body is a ReadableStream
const reader = response.body?.getReader();

for await (const chunk of readChunks(reader)) {
  console.log("CHUNK:", chunk);
}

await sleep();

CHUNK: Based on the context and chat history, you di
CHUNK: d not specify an explicit question before this inquiring request. The provided context features Andrew Ng's communication with students about their fields of study and coursework-related topics. Could you please clarify or provide additional context? I'm here to help!
